# Numba Performance exploration notebook

**Overview:** After realizing that removing numba speed up performance 8 fold, I wanted to make sure this persists for longer runtimes where JIT is designed to be effective.

In [2]:
import numba
import numpy as np

In [21]:
@numba.njit
def jitadd(x, y) -> np.ndarray:
    return (x * 0.2) + (y * 0.3)

def add(x, y) -> np.ndarray:
    return (x * 0.2) + (y * 0.3)

@numba.njit
def jitsub(x) -> np.ndarray:
    return x - (0.15 * x)

def sub(x) -> np.ndarray:
    return x - (0.15 * x)

In [13]:
input = np.ones(shape=(100, 100)) * np.random.random_sample((100, 100))
input

array([[0.25289281, 0.3012049 , 0.81652952, ..., 0.08600542, 0.59860995,
        0.7160782 ],
       [0.23049757, 0.7084206 , 0.008605  , ..., 0.12755473, 0.76453943,
        0.70370063],
       [0.34352444, 0.21580305, 0.24908259, ..., 0.54471826, 0.42611405,
        0.9940653 ],
       ...,
       [0.73328428, 0.46605497, 0.29684926, ..., 0.38692169, 0.1791475 ,
        0.24509559],
       [0.22376018, 0.713524  , 0.47561131, ..., 0.0983168 , 0.99624008,
        0.96001501],
       [0.17163253, 0.21930795, 0.68329088, ..., 0.34515248, 0.92341599,
        0.65903902]])

**No JIT**

In [22]:
%%timeit
TIMESTEPS = 1000
out_dict = {}
for i in range(TIMESTEPS):
    val = add(input, -input)
    out_dict[i] = sub(val)

61.3 ms ± 5.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Just the function being JIT**

In [24]:
%%timeit
TIMESTEPS = 1000
out_dict = {}
for i in range(TIMESTEPS):
    val = jitadd(input, -input)
    out_dict[i] = jitsub(val)

45.2 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**The functions and loop being JIT**

In [31]:
%%timeit
TIMESTEPS = 1000

@numba.njit
def loop(input_var, time_steps):
    out_dict = {}
    for i in range(time_steps):
        val = jitadd(input_var, -input_var)
        out_dict[i] = jitsub(val)
    return out_dict

out_dict = loop(input, TIMESTEPS)

477 ms ± 24.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
